##Parsing ANNOVAR

A new approach @CINECA

* Using [mETL](https://github.com/ceumicrodata/mETL) package
* Configuration file is in [YAML format](http://pyyaml.org/wiki/PyYAMLDocumentation) through python library

In [1]:
import yaml

### YAML configuration with python code

In [2]:
# Create
yfile = 'test.yml'
ifile = 'test_annovar.hg19.tsv'

data = { 
    'source': {
        'resource': ifile, 
        'source': 'TSV', 'headerRow': 0, 'skipRows': 1,
        'fields': [
            {'map': 'Chr', 'name': 'chromosome', 'type': 'String'},
        ]
    },
    'target': {
        'type': 'Database',
        'createTable': 'true', 'replaceTable': 'true', 'truncateTable': 'true',
        'table': 'test', 'url': 'sqlite:///test.db',
    }
}


In [3]:
# Write the file
with open(yfile, 'w') as outfile:
    outfile.write( yaml.dump(data, default_flow_style=False) )
# print
!head {yfile} -n 3

source:
  fields:
  - map: Chr


### Execute the parser

The following use the metl package to read the variants annotated and save it into a file database

In [4]:
# Prepare variables
bin = "metl"
# Execute via ipython
! {bin} {yfile} 

### Use the database

Read data in python and manipulate

In [5]:
import sqlite3 as lite

# Final notes


*Notes*

- transformation with python function
- manipulation (filters)
- convert from db to excel for download ;)